In [1]:
import numpy as np
import cv2 as cv
import utilsImg as u
import sklearn.cluster as sc
import scipy.cluster.hierarchy as slp
import segmentacion as s
from matplotlib import pyplot as plt

In [2]:
img_nir = cv.imread('datos/lechuga_ndvi/2.JPG',1)

In [3]:
b_showing = 1;
sh_scale =.25;
img_nir = u.imgResize(img_nir,sh_scale)
d_nir = img_nir[:,:,2]

In [ ]:
aux_rgb = cv.cvtColor(img_nir, cv.COLOR_BGR2RGB)
plt.imshow(aux_rgb)
plt.show()

In [ ]:
plt.close()

In [ ]:
def segmentacion_05(img,res): # img = imagen en escala de grises

    # lectura de imagen
    if img.dtype != np.uint8 :
        img = img * 255
        original = np.uint8(img)
    else:
        original = img


    #

    k_otsu = 11 if res==1 else  5;
    k_omorf = 6  if res==1 else 3;


    # Segmentar tierra y vegetación
    veg_mask = s.segOtsu(original, k_otsu)
    cv.imshow('veg_mask', veg_mask)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (k_omorf, k_omorf))
    veg_mask = cv.morphologyEx(veg_mask, cv.MORPH_CLOSE, kernel, iterations=1)
    cv.imshow('veg_mask2', veg_mask)
    # create a CLAHE object (Arguments are optional).
    #clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    #img_e = clahe.apply(original)
    img_e = cv.equalizeHist(original)
    img_veg = cv.bitwise_and(img_e, img_e, mask=veg_mask)
    cv.imshow('img_veg', img_veg)
    cv.waitKey()

    ##crear semilas
    dist = cv.distanceTransform(veg_mask , cv.DIST_L2, 3)  # distanceTransform(bw, dist, CV_DIST_L2, 3);
    cv.normalize(dist, dist, 0, 1., cv.NORM_MINMAX);
    dist = dist * 255
    dist = np.uint8(dist)
    cv.imshow('seem0', dist)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (k_omorf*2, k_omorf*2))
    seem = cv.morphologyEx(dist, cv.MORPH_ERODE, kernel, iterations=4)
    seem = cv.morphologyEx(seem, cv.MORPH_CLOSE, kernel, iterations=4)
    cv.imshow('seem1', seem)
    cv.waitKey(0)

    # apilcar watershe
    s.watershed2(seem, img_veg)
    
   
    return 0;

In [1]:
segmentacion_05(img_nir[:,:,2], res=0)

In [ ]:
cv.destroyAllWindows()